# The Philippine president's SONA

This scrapes the contents of https://www.officialgazette.gov.ph/past-sona-speeches/ for copies of the State of the Nation Addresses of Philippine presidents from 1936 to present.

The goal is to be able to use the SONAs for text analysis. Speeches are delivered before congress every fourth Monday of July and widely anticipated for setting the tone of a sitting administration.

An analysis is provided at the latter part of the notebook.

## Do your imports

In [1]:
import pandas as pd

import time
import re
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

from webdriver_manager.chrome import ChromeDriverManager

import requests
from bs4 import BeautifulSoup

## Allow Selenium to open up Chrome and automatically navigate through the website

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Could not get version for google-chrome with the any command: /Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome --version
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/104.0.5112.79/chromedriver_mac64.zip
Driver has been saved in cache [/Users/prinzmagtulis/.wdm/drivers/chromedriver/mac64/104.0.5112.79]
/var/folders/01/dz49lpcd4qq_yksvf114xzfc0000gn/T/ipykernel_1629/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
driver.get("https://www.officialgazette.gov.ph/past-sona-speeches/")

## Scraping proper: table

First step is to scrape all tabled information, that is, excluding all the contents of **links**.

In [4]:
rows= driver.find_elements(By.TAG_NAME, "tr")

We arrange the information into a **list of dictionaries** in preparation to transforming it into a **data frame** for pandas analysis later.

In [5]:
dataset=[]
for dicts in rows[1:]:
    data={}
    all_tds = dicts.find_elements(By.TAG_NAME, "td")
    if len(all_tds) == 5:
        prexy = data['president']= dicts.find_elements(By.TAG_NAME, "td")[0].text
        data['date']= dicts.find_elements(By.TAG_NAME, "td")[1].text
        data['title'] = dicts.find_elements(By.TAG_NAME, "td")[2].text
        try:
            data['link'] = dicts.find_elements(By.TAG_NAME, "a")[1].get_attribute('href')
        except:
            data['link'] = dicts.find_element(By.TAG_NAME, "a").get_attribute('href')
        data['venue'] = dicts.find_elements(By.TAG_NAME, "td")[3].text
        data['session'] = dicts.find_elements(By.TAG_NAME, "td")[4].text
        dataset.append(data)
    else:
        data['president'] = prexy
        data['date']= dicts.find_elements(By.TAG_NAME, "td")[0].text
        data['title'] = dicts.find_elements(By.TAG_NAME, "td")[1].text
        data['link'] = dicts.find_element(By.TAG_NAME, "a").get_attribute('href')
        data['venue'] = dicts.find_elements(By.TAG_NAME, "td")[2].text
        data['session'] = dicts.find_elements(By.TAG_NAME, "td")[3].text
        dataset.append(data)
dataset

[{'president': 'Manuel L. Quezon',
  'date': 'November 25, 1935',
  'title': 'Message to the First Assembly on National Defense',
  'link': 'http://www.officialgazette.gov.ph/1935/11/25/message-of-president-quezon-to-the-first-assembly-on-national-defense-november-25-1935/',
  'venue': 'Legislative Building, Manila',
  'session': 'First National Assembly, First Session'},
 {'president': 'Manuel L. Quezon',
  'date': 'June 16, 1936',
  'title': 'On the Country’s Conditions and Problems',
  'link': 'http://www.officialgazette.gov.ph/1936/06/16/manuel-l-quezon-second-state-of-the-nation-address-june-16-1936/',
  'venue': 'Legislative Building, Manila',
  'session': 'First National Assembly, First Session'},
 {'president': 'Manuel L. Quezon',
  'date': 'October 18, 1937',
  'title': 'Improvement of Philippine Conditions, Philippine Independence, and Relations with American High Commissioner',
  'link': 'http://www.officialgazette.gov.ph/1937/10/18/manuel-l-quezon-third-state-of-the-nation-

Our **first data frame**

In [6]:
df1 = pd.DataFrame(dataset)
df1.head()

,president,date,title,link,venue,session
0,Manuel L. Quezon,"November 25, 1935",Message to the First Assembly on National Defense,http://www.officialgazette.gov.ph/1935/11/25/m...,"Legislative Building, Manila","First National Assembly, First Session"
1,Manuel L. Quezon,"June 16, 1936",On the Country’s Conditions and Problems,http://www.officialgazette.gov.ph/1936/06/16/m...,"Legislative Building, Manila","First National Assembly, First Session"
2,Manuel L. Quezon,"October 18, 1937","Improvement of Philippine Conditions, Philippi...",http://www.officialgazette.gov.ph/1937/10/18/m...,"Legislative Building, Manila","First National Assembly, Second Session"
3,Manuel L. Quezon,"January 24, 1938",Revision of the System of Taxation,http://www.officialgazette.gov.ph/1938/01/24/m...,"Legislative Building, Manila","First National Assembly, Third Session"
4,Manuel L. Quezon,"January 24, 1939",The State of the Nation and Important Economic...,http://www.officialgazette.gov.ph/1939/01/24/m...,"Legislative Building, Manila","Second National Assembly, First Session"


## Scraping proper: actual speeches

We use BeautifulSoup on this one. The process is easier since we already have the links in the first df and all we have to do is to just **access and grab** their contents one by one.

I'm commenting this part out to avoid reading through a bunch of texts, but hey, it runs very well so try it on your own!

In [7]:
speeches=[]
for speech in dataset[0:]:
    href = speech['link']
    raw_html = requests.get(href).content
    doc = BeautifulSoup(raw_html, "html.parser")
    headers = doc.find_all(class_= 'large-9 large-centered columns')[1]
    text={}
    text['link']= speech['link']
    text['speech']= headers.text 
    speeches.append(text)
#speeches

As you can see, the speeches are arranged as a **single block** per row to match their place in the df. This is, of course, not the ideal way and may be improved. Below is a **second data frame** containing the links and speeches themselves.

We then **merge** this information with our earlier df.

In [8]:
df2=pd.DataFrame(speeches)
df2

,link,speech
0,http://www.officialgazette.gov.ph/1935/11/25/m...,\nMessage\nof\nHis Excellency Manuel L. Quezon...
1,http://www.officialgazette.gov.ph/1936/06/16/m...,\nMessage\nof\nHis Excellency Manuel L. Quezon...
2,http://www.officialgazette.gov.ph/1937/10/18/m...,\nMessage\nof\nHis Excellency Manuel L. Quezon...
3,http://www.officialgazette.gov.ph/1938/01/24/m...,\nMessage\nof\nHis Excellency Manuel L. Quezon...
4,http://www.officialgazette.gov.ph/1939/01/24/m...,\nMessage\nof\nHis Excellency Manuel L. Quezon...
...,...,...
79,https://www.officialgazette.gov.ph/2018/07/23/...,\n\n\n\nSTATE OF THE NATION ADDRESS OF \nRODRI...
80,https://www.officialgazette.gov.ph/2019/07/22/...,\n\n\n\nSTATE OF THE NATION ADDRESS OF \nRODRI...
81,https://www.officialgazette.gov.ph/2020/07/27/...,\n\n\n\n\n\n\n5TH STATE OF THE NATION ADDRESS ...
82,https://www.officialgazette.gov.ph/2021/07/26/...,\n\n\tState of the Nation Address of \n\tRodri...


Our final df.

In [9]:
merged = df1.merge(df2, suffixes=('_left'))
merged

/var/folders/01/dz49lpcd4qq_yksvf114xzfc0000gn/T/ipykernel_1629/4263006727.py:1: FutureWarning: Passing 'suffixes' as a <class 'str'>, is not supported and may give unexpected results. Provide 'suffixes' as a tuple instead. In the future a 'TypeError' will be raised.
  merged = df1.merge(df2, suffixes=('_left'))


,president,date,title,link,venue,session,speech
0,Manuel L. Quezon,"November 25, 1935",Message to the First Assembly on National Defense,http://www.officialgazette.gov.ph/1935/11/25/m...,"Legislative Building, Manila","First National Assembly, First Session",\nMessage\nof\nHis Excellency Manuel L. Quezon...
1,Manuel L. Quezon,"June 16, 1936",On the Country’s Conditions and Problems,http://www.officialgazette.gov.ph/1936/06/16/m...,"Legislative Building, Manila","First National Assembly, First Session",\nMessage\nof\nHis Excellency Manuel L. Quezon...
2,Manuel L. Quezon,"October 18, 1937","Improvement of Philippine Conditions, Philippi...",http://www.officialgazette.gov.ph/1937/10/18/m...,"Legislative Building, Manila","First National Assembly, Second Session",\nMessage\nof\nHis Excellency Manuel L. Quezon...
3,Manuel L. Quezon,"January 24, 1938",Revision of the System of Taxation,http://www.officialgazette.gov.ph/1938/01/24/m...,"Legislative Building, Manila","First National Assembly, Third Session",\nMessage\nof\nHis Excellency Manuel L. Quezon...
4,Manuel L. Quezon,"January 24, 1939",The State of the Nation and Important Economic...,http://www.officialgazette.gov.ph/1939/01/24/m...,"Legislative Building, Manila","Second National Assembly, First Session",\nMessage\nof\nHis Excellency Manuel L. Quezon...
...,...,...,...,...,...,...,...
79,Rodrigo Roa Duterte,"July 23, 2018",Third State of the Nation Address,https://www.officialgazette.gov.ph/2018/07/23/...,"Batasang Pambansa, Quezon City","Seventeenth Congress, Third Session",\n\n\n\nSTATE OF THE NATION ADDRESS OF \nRODRI...
80,Rodrigo Roa Duterte,"July 22, 2019",Fourth State of the Nation Address,https://www.officialgazette.gov.ph/2019/07/22/...,"Batasang Pambansa, Quezon City","Eighteenth Congress, First Session",\n\n\n\nSTATE OF THE NATION ADDRESS OF \nRODRI...
81,Rodrigo Roa Duterte,"July 27, 2020",Fifth State of the Nation Address,https://www.officialgazette.gov.ph/2020/07/27/...,"Batasang Pambansa, Quezon City","Eighteenth Congress, Second Session",\n\n\n\n\n\n\n5TH STATE OF THE NATION ADDRESS ...
82,Rodrigo Roa Duterte,"July 26, 2021",Sixth State of the Nation Address,https://www.officialgazette.gov.ph/2021/07/26/...,"Batasang Pambansa, Quezon City","Eighteenth Congress, Third Session",\n\n\tState of the Nation Address of \n\tRodri...


## Cleaning the data

### Remove 'applause' from speeches

These are portions of the speech that pertain to interruptions of applause from legislators during the actual speech. We remove this to avoid distortions on text analysis (i.e. on number of times it appears on a document). Alternatively, we can include "applause" as stopwords, but I am yet to figure out how to do that while keeping other parameters such as having Tagalog stopwords.

In [109]:
merged.speech = merged.speech.str.replace('applause', '', regex=False)
merged.speech = merged.speech.str.replace('Applause', '', regex=False)

# Initial analysis

## regex

We are now ready to take an **initial analysis** of the texts that we have. For this part, I provided some examples below using **regex**.

An important note on this method: the **str.contains** and **str.extractall** functions **ONLY** count *the number of speeches* that contain the word, *not how many times* the word was mentioned in the speech. We would look into the count of the words on the speeches later at a deeper analysis.

Words we ran here are based from peer-reviewed textual studies that gauge **populism.**

In [10]:
#Ran to just check the type of files we are dealing with.
merged.dtypes

president    object
date         object
title        object
link         object
venue        object
session      object
speech       object
dtype: object

### 'elite'

The word "elite" is found to have been often used by populist leaders. We find based on this initial analysis that in the case of Philippine presidents, three leaders (one of whom was **dictator** Ferdinand Marcos Sr.) were found to have included the word in their SONAs.

In [11]:
merged[merged.speech.str.contains(r"\belite", case=False, regex=True)].president.value_counts()

Ferdinand E. Marcos        2
Joseph Ejercito Estrada    1
Rodrigo Roa Duterte        1
Name: president, dtype: int64

In [12]:
pd.set_option('display.max_colwidth', None)
merged.speech.str.extractall(r'(.*\belite.+)', re.IGNORECASE)

,,0
,match,
31,0,"It is fortunate that the nation will, just two years from now, call a constitutional convention. I leave it to the delegates of that convention to evolve a truly democratic system, one which will not merely bend, as our system does today, to the wishes of a traditional elite and perpetuate the status quo. Democratic institutions must be instruments of national advancement. Democracy must symbolize change."
37,0,"Clearly, we face here the danger that our New Society is giving birth to a new government elite, who resurrect in our midst the privileges we fought in the past, who employ the powers of high office for their personal enrichment, as well as of their business colleagues, relatives, and friends."
60,0,"Our war on poverty is in the acceleration of the land redistribution processes under the agrarian reform program. We distributed more than 266,000 hectares of land to 175,000 landless farmers, including land owned by the traditional rural elite. [Applause]"
81,0,Great wealth enables economic elites and corporations to influence public policy to their advantage. Media is a powerful tool in the hands of oligarchs like the Lopezes who used their media outlets in their battles with political figures. I am a casualty of the Lopezes during the 2016 election.


### 'democracy' and 'demokrasya'

Dictator Ferdinand E. Marcos mentioned the word **"democracy"** in 10 of his SONAs followed by Gloria Arroyo (7 of 9 SONAs). In Filipino, Benigno Aquino III mentioned **"demokrasya"** in two of his six speeches. 



**Joseph Estrada**, whose term was cut short by a popular revolt in 2001, and **Rodrigo Duterte** mentioned the word in a single SONA. 

In [13]:
merged[merged.speech.str.contains(r"(.*\bdemocracy.+)", case=False, regex=True)].president.value_counts()

/var/folders/01/dz49lpcd4qq_yksvf114xzfc0000gn/T/ipykernel_1629/3410026565.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  merged[merged.speech.str.contains(r"(.*\bdemocracy.+)", case=False, regex=True)].president.value_counts()


Ferdinand E. Marcos        10
Gloria Macapagal-Arroyo     7
Manuel L. Quezon            5
Corazon C. Aquino           5
Fidel V. Ramos              5
Ramon Magsaysay             4
Diosdado Macapagal          4
Manuel Roxas                3
Elpidio Quirino             3
Carlos P. Garcia            2
Joseph Ejercito Estrada     1
Rodrigo Roa Duterte         1
Name: president, dtype: int64

In [14]:
merged[merged.speech.str.contains(r"(.*\bdemokrasya.+)", case=False, regex=True)].president.value_counts()

/var/folders/01/dz49lpcd4qq_yksvf114xzfc0000gn/T/ipykernel_1629/1026723483.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  merged[merged.speech.str.contains(r"(.*\bdemokrasya.+)", case=False, regex=True)].president.value_counts()


Benigno S. Aquino III      2
Ferdinand E. Marcos        1
Corazon C. Aquino          1
Gloria Macapagal-Arroyo    1
Name: president, dtype: int64

In [15]:
merged.speech.str.extractall(r'(.*\bdemocracy.+)', re.IGNORECASE).head(7)

0
  match                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
1 0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        In our day and generation democracy, as an effective system of government, is being challenged. Let this new democracy of ours show to the world that democracy can be as efficient as a dictatorship, without trespassing upon individual liberty and the sacred rights of the people.
2 0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      Still more: The Filipino workingman has heard, if he is not able to read, of the equality before the law of the poor and the rich. He has heard of democracy, liberty, and justice, since every candidate for an elective office discourses on these topics, painting to him in glowing terms the meaning of these words.
  1                                                                                                                                                                                                                                                                                                             One of the discoveries which we have made since the establishment of the Government of the Commonwealth is that, despite the large number of children that have gone through our public schools, as shown in the reports of the Bureau of Education, the literacy of the Islands has not increased proportionally, and the knowledge of those rudimentary subjects which the citizen of a democracy should have, has not been acquired by a population corresponding to the number of children that appear to have entered the public schools. The reason for this is simple. A large proprtion of the boys and girls who have been admitted to the schools have not remained long en

In [16]:
merged.speech.str.extractall(r'(.*\bdemokrasya.+)', re.IGNORECASE).head()

0
   match                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
40 0                            Nasa harap ng kapulungang ito ngayon ang katipunan ng mga hamon at pagsubok sa nakalipas na mga Kongreso, at ito na sana ang pangwakas na pagsubok kung makakaya natin gamitin ang demokrasya bilang mabisang sangkap ng katatagan at kaunlarang pambansa. Bagaman at kailangan pa ring magpatuloy ang pansamantalang pamahalaan, taglay ng kapulungang ito ang binhi ng matatag at masiglang lehislaturang tutugon sa ating pangangailangan kung ihahandog natin dito ang lahat ng ating talino at kakayahan.
   1      Tayo ngayon ay isang bansang pinalakas ng mga pagsubok na ating pinagdaanan, higit na nagkakaisa pagkaraan ng mga sigalutang dinanas, at higit na handa sa anumang uri ng pagsubok at suliranin. Natapos nating lampasan ang mahihigpit na balakid sa nakaraang lima-at-kalahating taon. Sa liwanag ng makabuluhang yugtong ito ng ating buhay bilang bansa at lahi, magagawa natin ang ating tungkuling pagtahak sa landas ng katuparan ng ating matayog na pangarap na pag-unlad, pagkakapantay-pantay, at ng tunay na demokrasya.
51 0                                                                                                                                                                                                                                                                                                                                                                                                Binigyang buhay ng mga Kabisig nating ito ang diwa ng ating Saligang Batas; binigyang halimbawa nila ang tunay na kahulugan ng demokrasya.
   1                                                                                                                                                                                May katiyakan ang ating tagumpay kung tayo’y magkakaisa. Kung kaya’t hinihimok ko kayo—kagalang-galang na mga Senador, Kongresista, at ang iba pang mga pinuno ng bayan—na muli tayong manumpa sa pangarap na nagbigkis sa atin noong 1986: ibalik at panatiliin ang demokrasya, kalayaan, karapatan, katatagang pangkabuhayan, at katarungang panlipunan.
65 0                                                                                                                                                                                                                                            Pinapangako ko ang isang bagong direksyon: mamamayan muna. Ang taong bayan ang pinakamalaki nating yaman. Ngunit madalas, kaunti lang ang atensyon na binibigay sa kanilang pag-unlad. Di tuloy matawid ang agwat ng mayaman at mahirap. Di tuloy mapa-abot sa lahat ang biyaya ng demokrasya.

## Segregating by president

We create separate dataframes from a select number of presidents to analyze using text analysis.

In [110]:
aquino = merged[(merged['president'] == 'Benigno S. Aquino III')] #Aquino
duterte = merged[(merged['president'] == 'Rodrigo Roa Duterte')] #Duterte
marcos = merged[(merged['president'] == 'Ferdinand E. Marcos')] #Marcos Sr.
erap = merged[(merged['president'] == 'Joseph Ejercito Estrada')] #Erap
marcosjr = merged[(merged['president'] == 'Ferdinand R. Marcos Jr.')] #Marcos Jr.

## Import text analysis libraries and identify paramaters

We will use Python's National Language Toolkit and [Scikit-Learn](https://scikit-learn.org/stable/index.html).

In [111]:
from sklearn.feature_extraction.text import CountVectorizer
import stopwordsiso as stopwords

In [112]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    return text #removes all numbers

In [113]:
y_columns = ['president', 'speeches']
BINARY=False
NGRAM_RANGE=(1,1)
MIN_DF=0
STPWORDS=stopwords.stopwords(["en", "tl"]) #removes Tagalog stopwords

vectorizer = CountVectorizer(
    stop_words=STPWORDS,
    ngram_range=NGRAM_RANGE,
    binary=BINARY,
    min_df=MIN_DF,
    preprocessor=preprocess_text
)

## Vectorizing

Simple counting of words that occur in a speech. We use the latest SONA here of the **current president Ferdinand Marcos Jr.** as an example.

In [114]:
X = vectorizer.fit_transform(marcosjr['speech'])
X

/Users/prinzmagtulis/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'daren', 'hadn', 'herse', 'himse', 'itse', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan'] not in stop_words.
  warnings.warn(


<1x1963 sparse matrix of type '<class 'numpy.int64'>'
	with 1963 stored elements in Compressed Sparse Row format>

In [115]:
marcos_vectors = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
# [print(x) for x in marcosjr.speech]
marcos_vectors.round(2)

,aagapay,aalaga,aalis,aaral,aatasan,abandon,ability,abolition,abono,abreast,abundance,academic,accelerated,accelerating,access,accessible,accomplish,accountability,accountable,accurate,acquired,acquisition,activities,activity,actual,adapt,address,addressed,addressing,adjusted,adjustments,administrasyon,administration,admission,adopt,adoption,adopts,advance,advancement,advantage,advertising,affairs,affordability,aforementioned,afternoon,agad,agarang,age,agencies,agency,agenda,aggressive,agile,agrarian,agree,agricultural,agriculture,agrikultura,agrikultural,ahensiya,ahensya,aics,aid,aim,aims,airport,airports,akma,akong,alagaan,alam,alert,alerts,alexander,alleged,alleviate,alliances,allied,allocation,ama,ambassadors,ambiguities,amend,amended,amending,amendments,amortization,anak,anchor,annual,annually,anong,antiquated,apatnapung,apostolic,appropriation,approved,arabia,aralan,araneta,araw,archives,arena,arisen,arising,armed,arroyo,artificial,arts,aspire,assembled,assist,assistance,assumptions,atomic,attached,attain,attainment,attract,attractive,augmented,authority,automate,availability,avenues,average,aversion,awarded,ayaw,ayuda,babagay,babuyan,backbone,bagong,bahagi,bakuna,balance,balansehin,band,banda,bank,bansa,banta,barko,barrel,based,basic,bata,batas,batasang,batid,bayan,bayanihan,begun,belief,belong,beloved,beneficial,beneficiaries,benefit,benefiting,benefits,benipisyaryo,beses,beset,bibilhin,bid,biktima,blood,blueprint,bonds,boost,booster,bot,bottleneck,bottlenecks,brand,branded,breadth,breakthrough,breed,bring,broad,broadband,brought,brown,budget,budgeted,budgeting,buhay,build,building,buildings,bukas,bukod,buksan,buksang,bulto,bumaba,bumalik,burahin,burden,bureau,bureaucracy,bureaus,bus,business,buwan,cabinet,calculated,called,capacity,capita,capital,capitalizing,carbon,care,careful,cash,catch,category,caused,cavite,cbs,cdc,cebu,center,centers,certifications,chain,chains,challenges,change,channel,characterized,charles,cheap,cheapest,cheers,chief,child,children,cities,citizen,citizens,city,civil,clarification,clarifying,class,classes,classrooms,climate,clinic,close,clouds,coherence,coinciding,cold,collaborate,collaboration,collection,college,collusion,combination,combined,command,commerce,commercial,commission,commit,commitment,committed,commodities,common,communicated,communications,communities,community,commuter,companies,compared,compete,competent,competition,competitive,complementary,complete,completed,complex,compliance,complications,comply,component,composed,comprehensive,computers,computing,concerned,concurrence,concurrent,condition,conditions,condonation,condoned,conduct,confidently,conflict,confronted,congress,connect,connecting,connectivity,consensus,consideration,considered,consistent,consolidation,consumer,consumption,context,continuation,continue,continued,continues,continuous,continuously,contracts,contributor,control,convenient,conventional,converting,cooperate,cooperation,cooperatives,coordinate,coordination,cornerstones,coronavirus,corp,corporate,corporation,corps,cost,counselling,countries,country,countrymen,courses,court,cover,coverage,covid,create,created,creates,creation,creative,creativity,credible,crisis,critical,crucial,crude,cultural,culture,current,custodians,customs,cut,daan,dagat,dagdag,dala,dalawang,dark,data,database,daunting,davao,day,dayuhang,deal,debt,decision,declarations,decongest,deeply,defense,deficit,degrees,delivered,delivery,demand,demonstrated,denr,department,departments,depensa,deploy,deployment,depth,derivations,deserve,desired,destination,determine,determining,develop,developed,developing,development,developments,devices,dict,difficult,digit,digital,digitalization,digitized,dilg,diminish,dinadagdagan,diplomatic,direct,directed,direst,disabilities,disability,disadvantaged,disaster,disbursement,disbursements,discipline,discussion,discussions,disease,disrupted,distinction,distinguished,distressed,distribution,diversifying,divide,doctors,document,doh,doktor,dollar,domestic,doors,dost

## Simple word count

In [116]:
pd.set_option('display.max_columns', None)
marcos_vectors = marcos_vectors.transpose()

In [117]:
marcos_vectors.columns = ['SONA1']
marcos_vectors.sort_values('SONA1', ascending=False).head(20)

,SONA1
natin,32
government,25
department,22
national,22
philippines,21
power,18
development,18
energy,17
upang,16
program,15


**Interpretation**: Based on this, the Tagalog words *sa* and *ang* were most commonly used, although these are **stop words** in Filipino and do not carry any meaning. There is a library to remove stop words in Tagalog for Python, but I have not explored it yet.

Let's use another example, this time from **President Rodrigo Roa Duterte**. We are now after the number of mentions of the word, **elite**.

### 'democracy' (and 'drug/s') in Duterte's speeches

In [118]:
X2 = vectorizer.fit_transform(duterte['speech'])
duterte_vectors = pd.DataFrame(X2.toarray(), columns=vectorizer.get_feature_names_out())
duterte_vectors.round(2)
duterte_vectors = duterte_vectors.transpose()

In [119]:
duterte_vectors.columns = ['SONA1', 'SONA2', 'SONA3', 'SONA4', 'SONA5', 'SONA6']
duterte_vectors.sort_values('SONA1', ascending=False).head(20)

,SONA1,SONA2,SONA3,SONA4,SONA5,SONA6
lang,52,42,1,34,15,49
iyan,50,4,0,1,8,13
government,32,47,21,29,42,35
wala,30,24,0,22,11,19
kasi,30,21,0,14,9,14
naman,28,20,0,8,7,13
time,22,33,8,26,22,36
yan,22,47,1,39,11,50
people,21,31,15,17,25,29
country,21,22,13,6,28,43


### Add a 'total' mention column

In [120]:
duterte_vectors['total'] = duterte_vectors.SONA1 + duterte_vectors.SONA2 + duterte_vectors.SONA3 + duterte_vectors.SONA4 + duterte_vectors.SONA5 + duterte_vectors.SONA6

In [121]:
duterte_vectors = duterte_vectors.sort_values('total', ascending=False)
duterte_vectors.head(20)

,SONA1,SONA2,SONA3,SONA4,SONA5,SONA6,total
government,32,47,21,29,42,35,206
lang,52,42,1,34,15,49,193
yan,22,47,1,39,11,50,170
yung,10,58,1,27,21,49,166
time,22,33,8,26,22,36,147
people,21,31,15,17,25,29,138
country,21,22,13,6,28,43,133
kayo,21,50,4,19,10,22,126
ninyo,19,39,0,22,7,24,111
wala,30,24,0,22,11,19,106


# TF-IDF

## Duterte speeches

In [146]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    stop_words=STPWORDS, 
    ngram_range=NGRAM_RANGE,
    binary=BINARY,
    min_df=MIN_DF,
    preprocessor=preprocess_text
)
X = vectorizer.fit_transform(duterte['speech'])
duterte_idf = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
#[print(x) for x in speeches.sentence]
duterte_idf.round(2)

/Users/prinzmagtulis/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'daren', 'hadn', 'herse', 'himse', 'itse', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan'] not in stop_words.
  warnings.warn(


aabot  aakyat  aalis  aambush  aano  aaway  abandon  abdul  abiding  \
0   0.00    0.00   0.00     0.00  0.01   0.01     0.00   0.00     0.00   
1   0.00    0.00   0.00     0.01  0.00   0.02     0.00   0.00     0.00   
2   0.00    0.00   0.00     0.00  0.00   0.00     0.00   0.00     0.00   
3   0.01    0.00   0.00     0.00  0.00   0.00     0.00   0.01     0.00   
4   0.00    0.00   0.01     0.00  0.00   0.00     0.00   0.00     0.01   
5   0.00    0.01   0.01     0.00  0.00   0.00     0.01   0.00     0.01   

   ability  abogado  abolish  abortion  abot  abra  abraham   abs  absent  \
0     0.01     0.00     0.00      0.00  0.00  0.00     0.00  0.00    0.00   
1     0.01     0.01     0.00      0.01  0.01  0.01     0.00  0.02    0.02   
2     0.00     0.00     0.00      0.00  0.00  0.00     0.02  0.00    0.00   
3     0.00     0.00     0.03      0.00  0.01  0.00     0.00  0.00    0.00   
4     0.02     0.01     0.00      0.00  0.00  0.00     0.00  0.01    0.00   
5     0.01     0.00     0.00      0.00  0.00  0.00     0.00  0.02    0.00   

   absinero  absolutely  absorb   abu  abugado  abuse  abuses  abutan  \
0      0.02        0.00    0.01  0.02     0.00   0.01    0.01    0.00   
1      0.00        0.02    0.00  0.00     0.00   0.00    0.00    0.00   
2      0.00        0.02    0.00  0.02     0.00   0.00    0.01    0.00   
3      0.00        0.00    0.00  0.00     0.01   0.00    0.02    0.01   
4      0.00        0.00    0.00  0.00     0.00   0.00    0.01    0.00   
5      0.00        0.00    0.01  0.00     0.00   0.02    0.00    0.00   

   academe  academy  acceded  accelerate  accelerated  accelerating  accept  \
0     0.00     0.00     0.00        0.02         0.00          0.00    0.02   
1     0.01     0.00     0.00        0.01         0.00          0.01    0.01   
2     0.00     0.00     0.00        0.00         0.00          0.00    0.00   
3     0.00     0.01     0.00        0.00         0.01          0.00    0.01   
4     0.00     0.01     0.00        0.00         0.00          0.00    0.00   
5     0.00     0.00     0.01        0.00         0.00          0.00    0.00   

   acceptable  acceptance  accepting  access  accessible  accident  \
0        0.01        0.01       0.00    0.03        0.01      0.00   
1        0.01        0.01       0.00    0.02        0.00      0.00   
2        0.00        0.00       0.00    0.03        0.01      0.00   
3        0.00        0.00       0.00    0.01        0.00      0.01   
4        0.00        0.00       0.00    0.01        0.01      0.00   
5        0.00        0.00       0.01    0.02        0.01      0.00   

   accommodate  accomplish  accomplished  accomplishments  accord  \
0         0.00        0.01          0.00             0.00    0.01   
1         0.00        0.00          0.00             0.00    0.00   
2         0.00        0.00          0.00             0.00    0.00   
3         0.00        0.00          0.01             0.03    0.00   
4         0.00        0.00          0.01             0.02    0.00   
5         0.02        0.00          0.00             0.03    0.00   

   accountability  accra  accredited  accumulated  accuracy  accurate  \
0            0.00   0.00        0.01         0.00      0.00      0.01   
1            0.00   0.00        0.00         0.00      0.00      0.00   
2            0.00   0.00        0.00         0.00      0.00      0.00   
3            0.00   0.00        0.00         0.00      0.01      0.00   
4            0.00   0.05        0.01         0.01      0.00      0.00   
5            0.02   0.00        0.01         0.00      0.00      0.00   

   achieve  achieved  achievements  achieving  acknowledge  acorn  acquired  \
0     0.01      0.00          0.00       0.01         0.00   0.00      0.00   
1     0.00      0.01          0.00       0.01         0.00   0.01      0.01   
2     0.01      0.00          0.00       0.00         0.00   0.00      0.00   
3     0.01      0.00          0.00       0.00         0.00   0.00      0.01

In [147]:
duterte_idf2 = duterte_idf.transpose()
duterte_idf2.columns = ['SONA1', 'SONA2', 'SONA3', 'SONA4', 'SONA5', 'SONA6']

In [148]:
duterte_idf2.sort_values('SONA5', ascending=False).head(20)

,SONA1,SONA2,SONA3,SONA4,SONA5,SONA6
government,0.168252,0.206161,0.220327,0.168924,0.256059,0.146750
pandemic,0.000000,0.000000,0.000000,0.000000,0.202722,0.131672
country,0.110415,0.096501,0.136393,0.034950,0.170706,0.180292
people,0.110415,0.135979,0.157377,0.099025,0.152416,0.121593
online,0.000000,0.000000,0.000000,0.000000,0.146410,0.007745
covid,0.000000,0.000000,0.000000,0.000000,0.146410,0.154909
congress,0.036805,0.057023,0.125901,0.133974,0.140223,0.109014
time,0.115673,0.144752,0.083934,0.151449,0.134126,0.150942
yung,0.052579,0.254412,0.010492,0.157274,0.128029,0.205449
oligarchy,0.000000,0.000000,0.000000,0.000000,0.123609,0.000000


In [181]:
duterte_slice = duterte_idf[['drug', 'drugs', 'mining', 'pandemic', 'covid']]
duterte_slice.sort_index().round(decimals=2)

,drug,drugs,mining,pandemic,covid
0,0.07,0.04,0.02,0.00,0.00
1,0.00,0.04,0.13,0.00,0.00
2,0.05,0.05,0.07,0.00,0.00
3,0.02,0.03,0.00,0.00,0.00
4,0.01,0.04,0.00,0.20,0.15
5,0.04,0.08,0.00,0.13,0.15


In [182]:
duterte_slice = duterte_slice.stack().reset_index()

In [183]:
duterte_slice = duterte_slice.rename(columns={'level_0': 'sona_no','level_1': 'term', 'tfidf': 'term', 0: 'tfidf'})

In [184]:
duterte_slice

,sona_no,term,tfidf
0,0,drug,0.073610
1,0,drugs,0.042063
2,0,mining,0.024601
3,0,pandemic,0.000000
4,0,covid,0.000000
5,1,drug,0.004386
6,1,drugs,0.035091
7,1,mining,0.129981
8,1,pandemic,0.000000
9,1,covid,0.000000


In [185]:
top_tfidf = duterte_slice.sort_values(by=['sona_no','tfidf'], ascending=[True,False]).groupby(['sona_no']).head(10)
top_tfidf

,sona_no,term,tfidf
0,0,drug,0.073610
1,0,drugs,0.042063
2,0,mining,0.024601
3,0,pandemic,0.000000
4,0,covid,0.000000
7,1,mining,0.129981
6,1,drugs,0.035091
5,1,drug,0.004386
8,1,pandemic,0.000000
9,1,covid,0.000000


## Chart it

In [187]:
import altair as alt

# # Terms in this list will get a red dot in the visualization
term_list = ['drug', 'drugs']

# adding a little randomness to break ties in term ranking
top_tfidf_plusRand = top_tfidf.copy()
top_tfidf_plusRand['tfidf'] = top_tfidf_plusRand['tfidf'] + np.random.rand(top_tfidf.shape[0])*0.0001

# base for all visualizations, with rank calculation
base = alt.Chart(top_tfidf_plusRand).encode(
    x = 'rank:O',
    y = 'sona_no:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["sona_no"],
)

# heatmap specification
heatmap = base.mark_rect().encode(
    color = 'tfidf:Q'
)

# red circle over terms in above list
circle = base.mark_circle(size=100).encode(
    color = alt.condition(
        alt.FieldOneOfPredicate(field='term', oneOf=term_list),
        alt.value('red'),
        alt.value('#FFFFFF00')        
    )
)

# text labels, white for darker heatmap colors
text = base.mark_text(baseline='middle').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('white'), alt.value('black'))
)

# display the three superimposed visualizations
(heatmap + circle + text).properties(width = 600, height=800)

alt.LayerChart(...)

## Marcos first SONA

In [130]:
vectorizer = TfidfVectorizer(
    stop_words=STPWORDS, 
    ngram_range=NGRAM_RANGE,
    binary=BINARY,
    min_df=MIN_DF,
    preprocessor=preprocess_text
)
X = vectorizer.fit_transform(marcosjr['speech'])
marcos_idf = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
#[print(x) for x in speeches.sentence]
marcos_idf.round(2)
marcos_idf = marcos_idf.transpose()

/Users/prinzmagtulis/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'daren', 'hadn', 'herse', 'himse', 'itse', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan'] not in stop_words.
  warnings.warn(


In [131]:
marcos_idf.columns = ['SONA1']
marcos_idf.sort_values('SONA1', ascending=False).head(20)

,SONA1
natin,0.252344
government,0.197144
department,0.173487
national,0.173487
philippines,0.165601
power,0.141944
development,0.141944
energy,0.134058
upang,0.126172
program,0.118286
